In [ ]:
import os
import csv
import glob
samples = []
for name in glob.glob("./data/*.csv"):
    print ( "Reading from logfile = " + name)
    with open(name)  as csvfile:
            reader = csv.reader(csvfile)
            for line in reader:
                samples.append(line)
                
print ( "Total samples found  : " + str(len(samples)))
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)


In [ ]:
import tensorflow as tf


In [ ]:
def  extractFileName ( abs_path):

    import os
    if os.name == "nt":
        split_char = '\\' 
    else:
        split_char = '/' 
    if '\\' in abs_path:
        # "  Windows Path " 
#         print ("windows path")
#         print (abs_path )
        image_name = './'+ abs_path.split ('\\')[-3] \
                            + split_char + abs_path.split ('\\')[-2] \
                            + split_char + abs_path.split ('\\')[-1]

    else:
        # "  Unix Path " 
        image_name = './'+ abs_path.split ('/')[-3] \
                            + split_char + abs_path.split ('/')[-2] \
                            + split_char + abs_path.split ('/')[-1]

    return image_name

In [ ]:
import cv2
img = cv2.imread("/data/center_image.jpg")
img = cv2.resize(img,(299,299))
crop_img = img[100:260, 0:299] # Crop from x, y, w, h -> 100, 200, 300, 400
# NOTE: its img[y: y + h, x: x + w] and *not* img[x: x + w, y: y + h]
import matplotlib.pyplot as plt
# # # Visualizations will be shown in the notebook.
%matplotlib inline

plt.imshow(crop_img)
plt.show()

In [ ]:
crop_img.shape

In [ ]:
train_samples_number_angle_no_zero = 0
for sample in train_samples:
    stearing_angle = float(sample[3])
    #if stearing_angle != 0:
    train_samples_number_angle_no_zero +=1
print ( "Total number of train samples in total= {}, but  with a stearing angle <> 0 = {} "\
   .format(len(train_samples),train_samples_number_angle_no_zero) )


In [ ]:
for sample in train_samples:
    left_image_name   = extractFileName( sample[1])
    print (left_image_name)

In [ ]:
import cv2
import numpy as np
import sklearn


def generator(samples, batch_size=256, shape = 299):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                
                left_image_name   = extractFileName( batch_sample[1])
#                 print (left_image_name)
                right_image_name  = extractFileName( batch_sample[2])
                
                center_image_name = extractFileName( batch_sample[0])
                
                left_image = cv2.imread(left_image_name)
                left_image = cv2.cvtColor (left_image, cv2.COLOR_BGR2RGB)
                left_image = cv2.resize(left_image,(shape,shape ))

                right_image = cv2.imread(right_image_name)
                right_image = cv2.cvtColor (right_image, cv2.COLOR_BGR2RGB)
                right_image = cv2.resize(right_image,(shape,shape ))

                center_image = cv2.imread(center_image_name)
                center_image = cv2.cvtColor (center_image, cv2.COLOR_BGR2RGB)
                center_image = cv2.resize(center_image,(shape,shape ))
    
                stearing_angle = float(batch_sample[3])
                #if stearing_angle != 0:
               # create adjusted steering measurements for the side camera images
                correction = 0.2 # this is a parameter to tune
                steering_left = stearing_angle + correction
                steering_right = stearing_angle - correction

                images.append(center_image)
                angles.append(stearing_angle)

                images.append(left_image)
                angles.append(steering_left)

                images.append(right_image)
                angles.append(steering_right)


#                 # augmented center image
#                 images.append(cv2.flip(center_image,1))
#                 angles.append(stearing_angle*-1.0)

#                 # augmented left image
#                 images.append(cv2.flip(left_image,1))
#                 angles.append(steering_left*-1.0)

#                 # augmented left image
#                 images.append(cv2.flip(right_image,1))
#                 angles.append(steering_right*-1.0)

            # end of the batch, yield the images gathered
            yield sklearn.utils.shuffle(np.array(images) , np.array(angles))
            #return sklearn.utils.shuffle(X_train ) #, y_train)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import tensorflow as tf

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x1 = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x2 = Dense(1024, activation='relu', name ="dense_2014")(x1)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, name ="final_1")(x2)

print ( " .. Completed ")

In [ ]:
 base_model.output


In [ ]:
x1


In [ ]:
from keras import layers

for i, layer in enumerate(base_model.layers):
    print(i, layer.name)
    #print ( layer.get_weights())
    #print ( layer.get_config())
    


In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Activation, Dropout, Reshape
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
import tensorflow as tf


model = Model(input=base_model.input, output=predictions)
print (" Model completed ")


In [ ]:

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='mse', optimizer='adam')

In [ ]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [ ]:
# train the model on the new data for a few epochs
batch_size=32
history_object = model.fit_generator(train_generator, steps_per_epoch= \
            train_samples_number_angle_no_zero/batch_size, validation_data=validation_generator, \
            validation_steps=len(validation_samples)/batch_size, epochs=3, verbose = 1)

In [ ]:

from keras.models import save_model

save_model(model, "model_inceptionv3_2.h5")
print ( "  ")
print ( " .. model saved to model_inceptionv3_2.h5 ")
print ( "  ")



In [ ]:

### print the keys contained in the history object
print(history_object.history.keys())

import matplotlib.pyplot as plt
# # # Visualizations will be shown in the notebook.
%matplotlib inline

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()


In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers[:249]):
   print(i, layer.name)


In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='mse')
#model.compile(loss='mse', optimizer='adam')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
batch_size=32
history_object = model.fit_generator(train_generator, steps_per_epoch= \
            train_samples_number_angle_no_zero/batch_size, validation_data=validation_generator, \
            validation_steps=len(validation_samples)/batch_size, epochs=3, verbose = 1)

In [ ]:
from keras.models import save_model

save_model(model, "model_inceptionv3_2.h5")
print ( "  ")
print ( " .. model saved to model_inceptionv3_2.h5 ")
print ( "  ")


In [ ]:

### print the keys contained in the history object
print(history_object.history.keys())

import matplotlib.pyplot as plt
# # # Visualizations will be shown in the notebook.
%matplotlib inline

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()


### show intermediate file

In [ ]:
import cv2
from keras.models import load_model
from keras.models import Model
print ( "Loading.... ")
modelobj = load_model('model_inceptionv3_2.h5')
print ( " ... model loaded")


In [ ]:
# Load test images
import cv2
import numpy as np
test_images = []

image = cv2.imread('./test_images/center1.jpg')
image = cv2.cvtColor (image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image,(64,64 ))   
image=  image[20:58, 0:64]
    
test_images.append(image)

# image = cv2.imread('./test_images/center2.jpg')
# image = cv2.cvtColor (image, cv2.COLOR_BGR2RGB)
# image = cv2.resize(image,(64,64 ))     
# test_images.append(image)


test_images = np.array(test_images)


In [ ]:
# Call the model to calculate an intermediate layer using the test images
layer_name = base_model.layers[10].name
intermediate_layer_model = Model(input=modelobj.input,
                                 output=modelobj.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(test_images)
intermediate_output.shape

In [ ]:
# Show the cropped images
depth = 0 
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 20))
new_image = []
plt.subplot(5, 5, 1 )
plt.axis('off')
plt.imshow(test_images[0])
i = 1
for single_output in intermediate_output:
        for image in single_output:
            i += 1
            plt.subplot(10, 5, i )
            plt.axis('off')
            single_output = single_output.astype(np.uint8)
            plt.imshow(image)
plt.show()    
